1) (5 points) Divide the data set into test/train sets (20/80)

In [2]:
import autograd.numpy as np
from autograd import grad 
import matplotlib.pyplot as plt

In [3]:
# load in dataset
import csv
fname = "hw4_naive.csv"
with open(fname) as f:
    reader = csv.reader(f, delimiter=",")

    data = []
    for row in reader:
        data.append(row)

header = data[0]
data = data[1:]

In [4]:
xs = [list(map(float, x[0:6])) for x in data]
ys = [int(x[6]) for x in data]

In [5]:
from sklearn.model_selection import train_test_split 
x_train, x_test, y_train, y_test = train_test_split(xs, ys, train_size=.8, test_size=.2)

2. Implement a multinomial naive bayes classifier **from scratch** with smoothing. (set default smoothing value to 1)

In [6]:
from copy import deepcopy

class MultinomialNaiveBayes():
    def __init__(self, xs, ys, classes, smoothing = 1):
        self.xs = xs
        self.ys = ys
        self.classes = classes
        self.class_counts = [0] * len(self.classes)
        self.smoothing = smoothing
        for y in ys:
            self.class_counts[self.classes.index(y)] += 1
    

    def predict(self, x):
        counts = [deepcopy([self.smoothing] * len(x)) for i in range(0, len(self.classes))]
        
        for x_vals, y in zip(self.xs, self.ys):
            i = self.classes.index(y)
            for j, x_v in enumerate(x_vals):
                if x_v == x[j]:
                    counts[i][j] += 1.0        
        joint = []

        for c in range(0, len(counts)):
            tot = 1
            for i in range(0, len(counts[c])):
                counts[c][i] /= (self.class_counts[c] + len(self.classes))
                tot *= counts[c][i]
            joint.append(tot)

        return self.classes[np.argmax(joint)]

3. Implement a Gaussian naive bayes classifier **from scratch** no smoothing.

In [7]:
from copy import deepcopy
from scipy.stats import norm

class GaussianNaiveBayes():
    def __init__(self, xs, ys, classes):
        self.sds = []
        self.means = []
        self.classes = classes
        filtered = [deepcopy([]) for i in range(0, len(classes))]
        for x, y in zip(xs, ys):
            i = self.classes.index(y)
            filtered[i].append(x)

        for i in range(0, len(classes)):
            sds = np.std(filtered[i], axis=0)
            means = np.mean(filtered[i], axis=0)
            self.sds.append(sds)
            self.means.append(means)

    def predict(self, x):
        joint = []

        for c in range(0, len(self.classes)):
            joint.append(1)
            for i in range(0, len(x)):
                sd = self.sds[c][i]
                mean = self.means[c][i]
                joint[c] *= norm.pdf(x[i], loc=mean, scale=sd)

        return self.classes[np.argmax(joint)]



4. Calculate the accuracy and the f1 score of test data using both of your models implemented above.

In [8]:
def evaluate(y_actual,y_pred):
    false_positive, false_negative, true_positive, true_negative = 0,0,0,0

    for y_a, y_p in zip(y_actual, y_pred):
        if y_a == y_p:
            if y_a == 1:
                true_positive+=1
            else:
                true_negative+=1
        else:
            if y_a == 1:
                false_negative+=1
            else:
                false_positive+=1

    return [false_positive, false_negative, true_positive, true_negative]

def acc(matrix):
    [_, _, tp, tn] = matrix
    return (tp + tn) / sum(matrix)

def f1(matrix):
    [fp, fn, tp, tn] = matrix
    prec = tp / (tp + fp)
    rec = tp / (tp + fn)

    return 2 * (prec * rec) / (prec + rec)


In [9]:
multi = MultinomialNaiveBayes(list(x_train), list(y_train), classes=[0,1])
y_pred = [multi.predict(x) for x in list(x_test)]

matrix = evaluate(y_test, y_pred)
print(f'Accuracy under NaiveBayes: {acc(matrix)}')
print(f'F1 Score under NaiveBayes: {f1(matrix)}')

Accuracy under NaiveBayes: 0.8955357142857143
F1 Score under NaiveBayes: 0.8764519535374867


In [10]:
from sklearn.naive_bayes import GaussianNB  

gauss = GaussianNaiveBayes(list(x_train), list(y_train), classes=[0,1])
y_pred = [gauss.predict(x) for x in list(x_test)]

matrix = evaluate(list(y_test), y_pred)
print(f'Accuracy under GaussBayes: {acc(matrix)}')
print(f'F1 Score under GaussBayes: {f1(matrix)}')

Accuracy under GaussBayes: 0.5973214285714286
F1 Score under GaussBayes: 0.31770045385779117


The following is to compare my code to sklearns

In [11]:
classifier = GaussianNB()
from sklearn.preprocessing import StandardScaler  
sc = StandardScaler()  
x_train = sc.fit_transform(x_train)  
x_test = sc.transform(x_test) 
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)

from sklearn.metrics import accuracy_score
print(f'Accuracy under GaussBayes: {accuracy_score(y_test, y_pred) }')

Accuracy under GaussBayes: 0.5955357142857143


------

## Extra Credit

### K-means

Implement a generalized K-means/median algorithm. You should have a single function that takes in as input the data points, K, and some other hyperparameters, specified below. The function should return K sets of data points. Each set corresponding to one cluster.
The hyperparameters your functions should support and the values they can take are:

- The method for calculating the centroid: Means or Median
- The initialization method: Random Split Initialization or Random Seed Selection Method
- Max_iter: max number of iterations to run the algorithm. 
- K: number of clusters


Note that your stopping condition should have two parts: 
1. stop if you reach the max iterations
2. stop if no change is made to the clusters in the last step.

You will be running this code in question 3 of the assignment. For this part you just need to implement the function.

In [12]:
def _calculate_centroids(buckets, centroid_meth):
    return [centroid_meth(list(bucket)) for bucket in buckets]

def k_means(data, centroid_meth, init_meth, max_iter, K):
    buckets = init_meth(data, K)

    changes = True
    i = 0
    while changes and i < max_iter:
        changes = False
        _centers = _calculate_centroids(buckets, centroid_meth)
        for j, bucket in enumerate(deepcopy(buckets)):
            for x in bucket:
                new_bucket = np.argmin([np.linalg.norm(_centers[k] - x) for k in range(0, len(_centers))])
                if new_bucket != j:
                    buckets[j].remove(x)
                    buckets[new_bucket].append(x)
                    changes = True
        i+=1
    return buckets

def random_split(data, K):
    groups = [deepcopy([]) for i in range(0, K)]
    for x in data:
        rand = int(np.random.random() * K)
        groups[rand].append(x)
    return groups

k_means({ 1,2,3,4,5,11,12,13,14,16}, np.mean, random_split, 2, 2)

[[12, 16, 11, 13, 14], [1, 2, 4, 3, 5]]

2. In this part of the assignment, you are implementing a function that calculates the SSE for a list of clusters. The function should take in a list of clusters (such as the output of the last function you implemented) and return a single SSE score.

In [13]:
def calculate_sse(buckets, centroid_meth):
    _centers = _calculate_centroids(buckets, centroid_meth)
    score = 0
    for j, bucket in enumerate(buckets):
        score += sum([np.linalg.norm(_centers[j] - x) for x in bucket])
    return score

buckets = k_means({ 1,2,3,4,5,11,12,13,14,16}, np.mean, random_split, 2, 2)
print(calculate_sse(buckets, np.mean))


13.2


3. Run the code you implemented in question 1 for k=2,3,4,5. Set the other hyperparameters to the following:

- The method for calculating the centroid: Mean
- The initialization method: Random Split Initialization 
- Max_iterations: 100

Calculate the SSE for each K using the function in question 2 and use these scores to pick the best K. What is the best K?

In [14]:

# load in dataset
import csv
fname = "hw4_cluster.csv"
with open(fname) as f:
    reader = csv.reader(f, delimiter=",")

    data = []
    for row in reader:
        data.append(row)

header = data[0]
data = data[1:]

In [15]:
xs_cluster = [list(map(float, x)) for x in data]

In [28]:
best = None
ks = [2,3,4,5]

def calc_all(k):
    buckets = k_means(xs_cluster, np.mean, random_split, 100, k)
    return calculate_sse(buckets, np.mean)

m = np.argmin(list(map(calc_all, ks)))

print(f'Minimum k = {ks[m]}')


Minimum k = 5


I also did kmeans using scipy to test mine!

In [45]:
from scipy.cluster import vq

best = None
ks = [2,3,4,5]

def calc_all(k):
    _, err = vq.kmeans(xs_cluster, k, 100)
    return err

m = np.argmin(list(map(calc_all, ks)))

print(f'Minimum k = {ks[m]}')


Minimum k = 5


I've decided to also implement HAC

In [ ]:
from __future__ import annotations
import itertools


class Cluster(object):
    def __init__(self, data = None, clusters = []):
        self._data = data
        self._clusters = clusters
        self._size = 1 if np.all(data) else sum(map(lambda x: x._size, self._clusters))

        self._centroid = data if np.all(data) else sum(map(lambda x: x._centroid * x._size, self._clusters)) / self._size

    def join(a: Cluster, b: Cluster, dist_met):
        new =  Cluster(clusters = [a, b])
        new._dist = dist_met(a, b)
        return new

    def __str__(self):
        if np.all(self._data):
            return f'Data: [{self._data}]'
        else:
            return f'Size: {self._size} Center: {self._centroid}'

    def euclid(a: Cluster, b: Cluster):
        return np.linalg.norm(a._centroid - b._centroid)
    
    def print(cluster: Cluster, prefix: str):
        
        if (np.all(cluster._data)):
            return prefix + '| > ' + str(cluster) + '\n'
        res = prefix + f'| - | ({cluster._dist})  \n' 
        for c in cluster._clusters:
            sub = Cluster.print(c, prefix + '| ')
            res += sub
        return res

def hac_clustering(data, dist_met):
    clusters = set(map(lambda x: Cluster(data = x), data))
    while (len(clusters) > 1):
        combos = list(itertools.combinations(clusters, 2))
        (c1, c2) = combos[np.argmin(list(map(lambda x: dist_met(*x), combos)))]
        c3 = Cluster.join(c1, c2, dist_met)
        clusters.remove(c1)
        clusters.remove(c2)
        clusters.add(c3)
    return clusters

cluster = hac_clustering(np.multiply(np.random.rand(10, 2), 100), Cluster.euclid).pop()
print(Cluster.print(cluster, ''))
    

| - | (67.75266079355055)  
| | - | (29.804434959936284)  
| | | > Data: [[84.90115724  2.13144185]]
| | | - | (26.787389070489745)  
| | | | > Data: [[50.59583955  7.39215512]]
| | | | > Data: [[68.1464232  27.62928008]]
| | - | (50.08360645674771)  
| | | - | (47.98589806274182)  
| | | | > Data: [[ 6.20618337 99.47471097]]
| | | | - | (41.06032018953776)  
| | | | | - | (25.322697091900487)  
| | | | | | - | (20.825682934389885)  
| | | | | | | - | (3.7611242792547435)  
| | | | | | | | > Data: [[37.9302068  63.15456551]]
| | | | | | | | > Data: [[38.10259131 59.39739378]]
| | | | | | | > Data: [[25.91185815 78.22263592]]
| | | | | | > Data: [[58.95882372 62.75654532]]
| | | | | > Data: [[ 9.5551327 38.5834121]]
| | | > Data: [[67.60532665 99.36749482]]

